In [1]:
import csv
import sys
import os
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg
import scipy.sparse.linalg as spla
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk

In [2]:
sys.path.append("../Python_code")
from reddit import *
from path import *

In [3]:
projects_to_remove = {'777', '1921', '1240', '1516', '1319'}
project_dict = dict()
with open('../data/atlas_filtered.json') as f:
    data = json.load(f)
projects_count = 0
for element in data['atlas']:
    pic_id = int(element["id"])
    if pic_id not in projects_to_remove:
        project_dict[pic_id] = projects_count
        projects_count += 1

In [4]:
users_dict = dict()
users_count = 0
with open("../data/tile_placements.csv",'r') as file:
    # Skip first line (header row)
    next(file, None)

    reader = csv.reader(file)
    
    for r in reader:
        user = r[1]
        if user not in users_dict:
            users_dict[user] = users_count
            users_count += 1

In [5]:
users_count

1166924

In [6]:
#1 - Position (i,j) is the number of pixels by user i in project j (final)
type1 = np.zeros((users_count, projects_count))
#2 - Position (i,j) is the number of updates by user i that agree with pixels in project j.
type2 = np.zeros((users_count, projects_count))
#3 - Position (i,j) is the number of updates by user i that in the bounding area of project j and disagree with that pixel in j. These are updates likely against the project.
type3 = np.zeros((users_count, projects_count))
        
        

In [7]:
with open("../data/tile_placements_proj.csv", "r") as file:
        reader = csv.reader(file, delimiter = ",")
         # Skip first line (header row)
        next(file, None)
        # user:1, pic_id: 5, pixel: 6, pixel_color: 7
        for r in reader:
            user = r[1]
            pic_id = int(r[5])
            pixel = int(r[6])
            pixel_color = int(r[7])
            
            try:
                if pic_id not in projects_to_remove:
                    if pixel == 1:
                        type1[users_dict[user],project_dict[pic_id]] += 1

                    if pixel_color == 1:
                        type2[users_dict[user],project_dict[pic_id]] += 1

                    elif pixel_color == 0:
                        type3[users_dict[user],project_dict[pic_id]] += 1
            except Exception as e:
                print(e)
                print(pic_id)
                print(user)
                break
            

In [ ]:
"""
    Type1 matrix: position (i,j) is the number of pixels by user i in project j 
    Create a new matrix where position (i,j) contains a tuple of the euclidean distance and 
    cosine distance between user i and user j.
"""

# Syntax:
# euc_dist = distance.euclidean(a,b)
# cos_dist = distance.cosine(a,b)
value = np.empty((), dtype = object)
value[()] = (0,0)
distance_matrix = np.full((users_count, users_count), value, dtype = object)
# distance_matrix = np.zeros((users_count, users_count), dtype=[('x', 'i4'), ('y', 'i4')])
for i in range(users_count):
    for j in range(i + 1, users_count):
        euc_dist = distance.euclidean( type1[:,i] , type1[:,j] )
        cos_dist = distance.cosine(type1[:,i] , type1[:,j] )
        distance_matrix[i,j] = (euc_dist, cos_dist)
